In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import scipy
import numpy as np
import pandas as pd
import os
os.chdir('/home/cat/code/manifolds')

from calcium import calcium
from wheel import wheel
from visualize import visualize
from tqdm import trange

import networkx as nx
import pandas as pd
from mne.viz import plot_connectivity_circle



import numpy as np
import scipy
from scipy.io import loadmat

from tqdm import tqdm, trange

Autosaving every 180 seconds


In [2]:
   
def compute_ensemble_occurance(c, selected_ensemble,
                              recompute=True):
    
    
    fname_out = os.path.join(c.root_dir,
                            c.animal_id,
                            c.session,
                            'suite2p',
                            'plane0',
                            'ensembles',
                            'ens_times.npy')
    
    if os.path.exists(fname_out)==False or recompute:
        ensemble_ids = np.arange(selected_ensemble.shape[0])

        #
        ens_times = []
        #ens_network = []
        for ensemble_id in tqdm(ensemble_ids):
            ww = np.zeros(rasters.shape[1])
            idx = selected_ensemble[ensemble_id].squeeze()
            #print (idx)
            ww[idx] = 1

            W = np.outer(ww,ww)
            #print ("W: ", W.shape)
            # set single occurances to zero!
            np.fill_diagonal(W,0)
            #print ("W: ", W.shape)
            
            #
            f = np.zeros(Zib.shape[0])
            s = np.zeros(Zib.shape)
            for t in range(Zib.shape[0]):
                temp1 = np.matmul(Zib[t].T,W)
                temp2 = np.matmul(temp1,Zib[t])
                f[t]=temp2

#                 #
#                 for i in idx:
#                     if Zib[t,i]>=thresh:
#                         s[t,i]=1

            #
            ens_times.append(f)
            #ensemble_stacks.append(s)
        
        np.save(fname_out, ens_times)
        
        ens_times = np.array(ens_times)
    else:
        ens_times = np.load(fname_out)
            
        
    return ens_times
        
        
def plot_ensemble_occurances(ens_times, 
                            w):     
        
    # downsample track times to match data that went into Matlab ICA routines
    factor = 15
    vel_down = w.track.velocity.values.reshape(-1,factor).mean(axis=1)
    vel_times = w.track.velocity.times.reshape(-1,factor).mean(axis=1)
    print (vel_down.shape)

    #
    ctr=0
    #fig=plt.figure()
    ensemble_times = []
    ensemble_stacks = []
    thresh = 0

    y_scale = 40000

    if False:
        plt.plot(times, vel_down*2E5-y_scale*2,
            c='black',
            linewidth=2)
    else:
        plt.plot(w.track.velocity.times,
                 w.track.velocity.values*2E5-y_scale*2,
            c='black',
            linewidth=2)

    times = vel_times
    ensemble_ids = np.arange(ens_times.shape[0])
    
    print ("# of ensembels: ", ensemble_ids)
    
    for k in range(ensemble_ids.shape[0]):

        f = ens_times[k]        
        # 
        if True:
            plt.plot(times,f+ctr*y_scale)
        else:
            plt.plot(times, np.log(f+0.0001)+ctr*25)
        plt.xlabel("Time (sec)")
        plt.ylabel("Ensemble ID")

        # plot shaded version also
        if True:
            plt.plot(w.track.velocity.times,
                 w.track.velocity.values*5E4+ctr*y_scale,
            c='black',
            alpha=.2,)



        ctr+=1


    ylabels1 = np.arange(0,ctr*y_scale, y_scale)
    ylabels2 = np.arange(len(ensemble_ids))
    plt.yticks(ylabels1,ylabels2)
    #
    #plt.yticks([])
    plt.show()

def plot_ensemble(row,col,ens_id,ax_ctr,
                 selected_ensemble,
                 c,
                 graphs):
    
    ax=plt.subplot(row,col,ax_ctr+1)
    plt.xticks([])
    plt.yticks([])

    ids = selected_ensemble[ens_id].squeeze()
    #print (ids)
    for id2 in ids:
        contour = c.get_footprint_contour(id2)
        plt.plot(contour[:,1],
                contour[:,0],
                c='black')

    cmap = plt.get_cmap("viridis", 10)
    # normalize connectivity within each graph
    graph1 = graphs[ens_id]
    if True:
        graph1 = (graph1-np.min(graph1))/(np.max(graph1)-np.min(graph1))#+0.01
        
    # scale the colros up
    if True:
        graph1*=1
        graph1 = np.clip(graph1,0,.5)
    #else:
    #    graph1 = np.log(graph1+1.0)
        
        
    for k in trange(graph1.shape[0]):
        for p in range(k+1, graph1.shape[0],1):
            centre1 = c.get_footprint_contour(ids[k]).mean(0)
            centre2 = c.get_footprint_contour(ids[p]).mean(0)
            cb = plt.plot(
                    [centre1[1], centre2[1]],
                    [centre1[0], centre2[0]],
                    linewidth = graph1[k,p]*10,
                    #color='red',
                    color=cmap(int(graph1[k,p]*20)),
                    alpha=.2
                    )
            
                # 
    sm = plt.cm.ScalarMappable(cmap=cmap, 
                               norm=plt.Normalize(vmin=0, vmax=.5))
    plt.colorbar(sm)



    
    plt.xlim(0,512)
    plt.ylim(0,512)

    return ax, cb

   
def compute_ensemble_graph(c, selected_ensemble):
    
    
    fname_out = os.path.join(c.root_dir,
                            c.animal_id,
                            c.session,
                            'suite2p',
                            'plane0',
                            'ensembles',
                            'ens_graphs.npy')
    
    if os.path.exists(fname_out)==False:
        
        factor=15
        Z = c.F_upphase_bin.T
        Z_down = []
        for k in range(0,Z.shape[0],factor):
            temp = Z[k:k+factor]
            Z_down.append(temp.max(0))
            
        Z_down = np.vstack(Z_down)
        print (Z_down.shape)
        
        ensemble_ids = np.arange(selected_ensemble.shape[0])

        #
        ens_graphs = []
        for ensemble_id in tqdm(ensemble_ids):
            #ww = np.zeros(rasters.shape[1])
            idx = selected_ensemble[ensemble_id].squeeze()
            #print (ensemble_id, idx.shape)

            graph = np.zeros((idx.shape[0],
                             idx.shape[0]))
            
            # 
            for t in range(Z_down.shape[0]):
                temp = Z_down[t]
                #print ("time slice; ", temp)
                temp2 = temp[idx]
                #print ("indexed time slice; ", temp)
                idx2 = np.where(temp2>0)[0]
                
                for i in range(idx2.shape[0]):
                    for p in range(i+1,idx2.shape[0],1):
                        graph[idx2[i],idx2[p]]+=1
                        #print ("adding")

            #
            ens_graphs.append(graph)
            #ensemble_stacks.append(s)
        
        np.save(fname_out, ens_graphs)
    else:
        ens_graphs = np.load(fname_out)
            
            
    return ens_graphs, Z_down



In [3]:
##########################################################
############ ALTERNATIVE: GET RASTERS FROM DF_F ##########
##########################################################
root_dir = '/media/cat/4TB/donato/'
animal_id = 'DON-006084'
session = '20210519'

binarization_method='upphase'

#
c = calcium.Calcium()
c.root_dir = root_dir
c.animal_id = animal_id
c.session = session
c.load_suite2p()

# 
c.load_binarization()
if binarization_method=='onphase':
    traces = c.F_onphase_bin
elif binarization_method=='upphase':
    traces = c.F_upphase_bin
else:
    print ("METHOD NOT FOUND")


c.load_footprints()
    
print ("binarized data: ", traces.shape)

w = wheel.Wheel()
w.root_dir = os.path.join(root_dir,
                          animal_id,
                          session,
                          'TRD-2P')
w.animal_id = animal_id
w.session = session
w.load_track()
w.compute_velocity()

    
# these are binned rasters use specifically for Matlab SGC/ICA ensemble detection
rasters = np.load(os.path.join(root_dir,
                              animal_id,
                              session,
                              'suite2p',
                              'plane0',
                              'ensembles',
                              'F_upphase_binned_sum.npy'))
print ("Rasters (time:#cells): ", rasters.shape)

# if False:
#     plt.imshow(rasters.T,
#               aspect='auto',
#               #vmin=0,
#               #vmax=1
#                interpolation='none'
#               )
#     plt.show()



(531, 55740)
binarized data:  (531, 55740)
Rasters (time:#cells):  (3716, 531)


In [4]:
#######################################################
################ Z-SCORE RASTERS ######################
#######################################################
Zib = np.zeros(rasters.shape)
print ("Zib: ", Zib.shape)

# compute standard deviations
sigma = np.std(rasters,axis=0)
print ("sigma: ", sigma.shape)

# compute means
means = np.mean(rasters,axis=0)
print ("means: ", means.shape)

# compute normalized projectuion
for t in trange(Zib.shape[0]):
    for q in range(Zib.shape[1]):
        Zib[t,q]= (rasters[t,q]-means[q])/sigma[q]
    


Zib:  (3716, 531)
sigma:  (531,)
means:  (531,)


100%|█████████████████████████████████████████████████████████████████| 3716/3716 [00:02<00:00, 1812.83it/s]


In [12]:
#######################################################
################## LOAD ASSEMBLIES ####################
#######################################################
fname =  os.path.join(root_dir,
                      animal_id,
                      session,
                      'suite2p',
                      'plane0',
                      'ensembles',
                      'Upphase_sum_binned_ICA_ASSEMBLIES.mat')

# 
data = loadmat(fname)
#print (data.keys())
ens_mp = data['mp_assemblies'].squeeze()-1
ens_cs = data['cs_assemblies'].squeeze()-1

# 
print ("Cs ensembles: ", ens_cs.shape)
print ("Mp ensembles: ", ens_mp.shape)
fname =  os.path.join(root_dir,
                      animal_id,
                      session,
                      'suite2p',
                      'plane0',
                      'ensembles',
                      'Upphase_sum_binned_SGC-ASSEMBLIES.mat')

data = loadmat(fname)
#print (data.keys())
assemblies = data['assemblies'].squeeze()-1
assembly_pattern_detection = data['assembly_pattern_detection'].squeeze()

print ("SGC assembiles ", assemblies.shape)
#print ("assembly_pattern_detection: ", assembly_pattern_detection.shape)
    
    
#
#w = np.zeros(rasters.shape[1])


Cs ensembles:  (25,)
Mp ensembles:  (65,)
SGC assembiles  (8,)


In [24]:
######################################################
######## GET ENSEMBLE OCCURANCES AND VISUALIZE #######
######################################################
# NOTE need to delete previous ens_times.npy file to reprocess new data
selected_ensembles = ens_cs.copy()
#selected_ensembles = ens_mp.copy()
#selected_ensembles = assemblies.copy()

print ("selected_ensembles: ", selected_ensembles.shape)
recompute = True
ens_times= compute_ensemble_occurance(c, selected_ensembles,
                                      recompute)
fig=plt.figure(figsize=(20,20))
plot_ensemble_occurances(ens_times, w)

if False:
    plt.savefig('/home/cat/ensembles.svg')
    plt.close()
else:
    plt.show()

selected_ensembles:  (25,)


100%|███████████████████████████████████████████████████████████████████████| 25/25 [00:08<00:00,  2.82it/s]


(3716,)
# of ensembels:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24]


In [33]:
######################################################
######## GET ENSEMBLE OCCURANCES AND VISUALIZE #######
######################################################

#
ensemble_id = 13
neuron_ids = selected_ensembles[ensemble_id].squeeze()
print ("neuron ids: ", neuron_ids)

#
print (ens_times[ensemble_id].shape)
temp = ens_times[ensemble_id]
median = np.median(ens_times)
idx = np.where(temp>median)[0]

idx = np.arange(temp.shape[0])

# 
print ("rasters: ", rasters.shape)
rasters_ensemble = rasters[idx][:, neuron_ids]
print (rasters.shape, rasters_ensemble.shape)

sums = rasters_ensemble.sum(0)
print (sums.shape)
idx2 = np.argsort(sums)[::-1]
rasters_ensemble = rasters_ensemble[:,idx2]


fig=plt.figure()
# ax=plt.subplot(211)
# plt.plot(ens_times[ensemble_id])

# extract ensemble occurance time series
temp2 = ens_times[ensemble_id,idx]
idx3 = np.argsort(temp2)[::-1]
xlabels = [
    #'Time bins in which ensemble is active (0.5 sec bins)',
    'Time bins (0.5 sec bins)',
           "Reordered Time bin (0.5 sec bins)"]

# extract binarized mua for all celss
temp3 = rasters_ensemble.copy()
temp3 = np.clip(temp3, 0,1)
mua_binarized = np.sum(temp3, axis=1)
print ("mua_binarized.sh", mua_binarized.shape)




if False:
    for p in range(2):
        ax=plt.subplot(2,1,p+1)

        if p==1:
            rasters_ensemble = rasters_ensemble[idx3]
            temp2 = temp2[idx3]

        plt.xlabel(xlabels[p])
        temp2 = temp2/np.max(temp2)*rasters_ensemble.shape[1]
        plt.plot(temp2, 
                c='red')

        #
        plt.imshow(rasters_ensemble.T,
                  aspect='auto',
                  extent=[0,rasters_ensemble.shape[0], 0, rasters_ensemble.shape[1]],
                  interpolation='none'
                  )


        plt.ylabel("Neuron Id within the ensemble")
else:
    ax=plt.subplot(1,1,1)

    plt.xlabel(xlabels[0])
    temp2 = temp2/np.max(temp2)*rasters_ensemble.shape[1]
    plt.plot(temp2, 
            c='red',
            alpha=.5)
    
    
    mua_binarized = mua_binarized/np.max(mua_binarized)*rasters_ensemble.shape[1]
    plt.plot(mua_binarized,
            c='white',
            alpha=.5)

    #
    plt.imshow(rasters_ensemble.T,
              aspect='auto',
              extent=[0,rasters_ensemble.shape[0], 0, rasters_ensemble.shape[1]],
              interpolation='none'
              )


    plt.ylabel("Neuron Id within the ensemble")        
        
plt.suptitle("Total neurons: "+str(rasters.shape[1])+ ", selected ensemble ID: "+str(ensemble_id))

plt.show()


neuron ids:  [ 20  24  56  84  99 113 120 139 140 148 149 195 217 258 287 324 341 345
 366 386 406 443 452 458 489 491 527]
(3716,)
rasters:  (3716, 531)
(3716, 531) (3716, 27)
(27,)
mua_binarized.sh (3716,)


In [29]:
######################################################
######## BUILD ENSEMBLE GRAPH BASED ON OCCURANCE #####
######################################################
#selected_ensembles = assemblies.copy()

graphs, Z_down = compute_ensemble_graph(c, selected_ensembles)
    


(3716, 531)


100%|███████████████████████████████████████████████████████████████████████| 25/25 [00:01<00:00, 23.16it/s]
/home/cat/anaconda3/envs/manifolds/lib/python3.8/site-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [30]:
###############################################
############# SPATIAL GRAPH ############
###############################################

# plot all ensembles
if False:
    ens_ids = np.arange(25)
    fig=plt.figure(figsize=(40,20))
    row=5
    col=5
    
# plot only selected ones
else:
    ens_ids = [0,4]
    row=1
    col=1
    

for ens_id in tqdm(ens_ids):
    fig=plt.figure() # move this outside for all cells
    
    #ax_ctr=end_ids
    ax_ctr=0
    
    ax, cb = plot_ensemble(row,col,ens_id,ax_ctr,
                 selected_ensemble,
                 c,
                 graphs)

    plt.tight_layout()
    plt.show()


#plt.suptitle("ICA-CS Ensembles")
if False:
    plt.savefig('/home/cat/ensmbles.svg')
    plt.close()
    
else:
    plt.show()


100%|█████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.58it/s]


In [31]:
########################################
############ NETWORK GRAPHS ############
########################################

import networkx as nx

ids = np.arange(len(graphs))
#fig = plt.figure(figsize=(40,20))
ids = [14,19]
for i in ids:
    #plt.subplot(5,5,i+1)
    fig=plt.figure()
    G = nx.Graph()
    graph = graphs[i]
    
    if True:
        graph = (graph-np.min(graph))/(np.max(graph)-np.min(graph))/0.4
#     if True:
#         graph = graph#**3
        
    # 
    if False:
        ax1=plt.subplot(1,2,1)
        y=np.histogram(graph, bins=np.arange(0,1.05,0.05))
        plt.plot(y[1][:-1],y[0])
        plt.semilogy()
    
    #ax2=plt.subplot(1,2,2)
    if False:
        idx = np.where(graph<0.05)
        graph[idx]=0
    
    print ("Graph: ", graph.shape)
    for k in range(graph.shape[0]):
        for p in range(k+1, graph.shape[0],1):
            G.add_edge(k,p,color='r',weight=graph[k,p])

    colors = nx.get_edge_attributes(G,'color').values()
    weights = nx.get_edge_attributes(G,'weight').values()

    if True:
        nx.draw(G, edge_color=colors, 
            width=list(weights),
            with_labels=True,
            node_color='grey',
            node_size=0,
            font_size=10
               )
    else:
        pos = nx.circular_layout(G)
        nx.draw(G, pos, 
            edge_color=colors, 
            width=list(weights),
            with_labels=True,
            node_color='grey')
        
        
    plt.title(str(i))
    
    
if False:
    plt.savefig('/home/cat/net.svg')
    plt.close()
else:
    plt.show()

Graph:  (22, 22)
Graph:  (30, 30)


In [319]:
 ########################################

import networkx as nx

ids = np.arange(len(graphs))
#fig = plt.figure(figsize=(40,20))
ids = [14,19]
for i in ids:
    #plt.subplot(5,5,i+1)
    fig=plt.figure()
    G = nx.Graph()
    graph = graphs[i]
    
    if True:
        graph = (graph-np.min(graph))/(np.max(graph)-np.min(graph))/0.4
    if True:
        graph = graph#**3
        
    # 
    ax1=plt.subplot(1,2,1)
    y=np.histogram(graph, bins=np.arange(0,1.05,0.05))
    plt.plot(y[1][:-1],y[0])
    plt.semilogy()
    
    ax2=plt.subplot(1,2,2)
    if False:
        idx = np.where(graph<0.05)
        graph[idx]=0
    
    print ("Graph: ", graph.shape)
    for k in range(graph.shape[0]):
        for p in range(k+1, graph.shape[0],1):
            G.add_edge(k,p,color='r',weight=graph[k,p])

    colors = nx.get_edge_attributes(G,'color').values()
    weights = nx.get_edge_attributes(G,'weight').values()

    pos = nx.circular_layout(G)
    if True:
        nx.draw(G, edge_color=colors, 
            width=list(weights),
            with_labels=True,
            node_color='grey')
    else:
        nx.draw(G, pos, 
            edge_color=colors, 
            width=list(weights),
            with_labels=True,
            node_color='grey')
    plt.title(str(i))
    
    
if False:
    plt.savefig('/home/cat/net.svg')
    plt.close()
else:
    plt.show()

Graph:  (15, 15)
Graph:  (27, 27)


In [283]:
# ########################################
# ########################################
# ########################################

# # plot all ensembles
# if True:
#     ens_ids = np.arange(25)
#     fig=plt.figure(figsize=(40,20))
#     row=5
#     col=5
    
# # plot only selected ones
# else:
#     ens_ids = [0,4]
#     row=1
#     col=1
    

# fig=plt.figure() # move this outside for all cells
# for ens_id in tqdm(ens_ids):
#     ax=plt.subplot(5,5,ens_id+1)

#     N = graphs[ens_id].shape[0]  # Number of nodes
#     node_names = [f"{i}" for i in range(N)]  # List of labels [N]

#     # Random connectivity
#     con = np.float64(graphs[ens_id])
#     con = con + con.T
#     con = (con-np.min(con))/(np.max(con)-np.min(con))#*100

#     plot_connectivity_circle(con, node_names)   

# #plt.suptitle("ICA-CS Ensembles")
# if False:
#     plt.savefig('/home/cat/ensmbles.svg')
#     plt.close()
    
# else:
#     plt.show()
    
    

 72%|█████████████████████████████████████████████████████████████████████████████▊                              | 18/25 [00:05<00:02,  3.40it/s]/home/cat/anaconda3/envs/manifolds/lib/python3.8/site-packages/mne/viz/circle.py:299: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(8, 8), facecolor=facecolor)
 92%|███████████████████████████████████████████████████████████████████████████████████████████████████▎        | 23/25 [00:06<00:00,  3.43it/s]


KeyboardInterrupt: 

In [284]:
# import pylab
# import networkx as nx
# #G=nx.cycle_graph(80)
# id3 = 2
# #G = nx.from_numpy_matrix(graphs[id3])
# ax=plt.subplot(121)

# G = nx.from_numpy_matrix(graphs[id3])

# # 
# layout = nx.spring_layout(G)
# nx.draw(G, layout, node_size=1000, with_labels=True, font_weight='bold',    font_size=15)
# labels = nx.get_edge_attributes(G,'weight')

# nx.draw_networkx_edge_labels(G,
#                              pos=layout,
#                              edge_labels=labels)



# pos=nx.circular_layout(G)
# pylab.figure(1)
# nx.draw(G,pos)

# ax=plt.subplot(122)
# plt.imshow(graphs[id3])


# # pylab.figure(2)
# # nx.draw(G,pos,node_size=60,font_size=8)
# # pylab.figure(3,figsize=(12,12))
# # nx.draw(G,pos)
# pylab.show()

Graph:  (20, 20)
Graph:  (32, 32)


In [77]:
######################################################
################## LOOK AT EACH ######################
######################################################
id_ = 0
sample_rate=2

######################
fig=plt.figure()
ax=plt.subplot(3,1,1)
e = ensemble_times[id_]
plt.plot(times,e)

# 
plt.xlim(0,e.shape[0]/sample_rate)
plt.title("Ensemble: "+str(id_))
plt.ylabel("Ensemble presence (projection)")
plt.ylim(0,10000)


##################
ax=plt.subplot(3,1,2)
ee = ensemble_stacks[id_]
plt.imshow(ee.T,
          aspect='auto',
          interpolation='none',
          extent=[0,ee.shape[0]/sample_rate, 0,ee.shape[1]])
plt.xlabel("Time (sec)")
plt.ylabel("Neuron ID")

##################
ax=plt.subplot(3,1,3)
ee = ensemble_stacks[id_]
sums2 = ee.sum(axis=1)
idx = np.where(sums2>0)[0]
print (idx)
temp = rasters[idx]
print (temp.shape)
plt.imshow(temp,
          aspect='auto',
          interpolation='none',
          extent=[0,temp.shape[0]/sample_rate, 
                  0,temp.shape[1]]
          )
plt.xlabel("Time - discontinuous (sec)")
plt.ylabel("Removed zero-activity frames\nNeuron ID")

plt.show()


[   0    1    2 ... 3713 3714 3715]
(2349, 531)


In [93]:
##################################################
################ LOAD FOOTPRINTS #################
##################################################
root_dir = '/media/cat/4TBSSD/donato/binarized_traces/'
animal_id = ''
session = ''

#
c = calcium.Calcium()
c.animal_id = animal_id
c.root_dir = root_dir
c.session = session
c.recompute = False
c.load_suite2p()

#
c.load_binarization()

# 
c.load_footprints()




(531, 55740)


In [95]:
######################################################
################ VISUALIZE ENSEMBLES #################
######################################################
selected_ensemble = ens_cs.copy()
ensemble_ids = np.arange(selected_ensemble.shape[0])

ensemble_ids = np.arange(8)
cmap = plt.get_cmap("viridis", Zib.shape[1])

fig=plt.figure()

ctr2=0
for ensemble_id in ensemble_ids:
    ax=plt.subplot(3,3,ctr2+1)
    print ("ensemble stack: ", ensemble_times.shape) 
    ens_times = ensemble_times[ensemble_id]
    #print (ens_times.shape)

    idx = np.where(ens_times>50)[0]
    print ("# of occurances: of ensemble over threshold", idx.shape)

    # 
    norms = np.zeros(Zib.shape[1])
    print ("norms: ", norms.shape, "Zib: ", Zib.shape)

    # 
    for ctr,i in enumerate(idx):
        activity = Zib[i]
        idx2 = np.where(activity>0)[0]
        norms[idx2]+=1

    norms = (norms-np.min(norms))/(np.max(norms)-np.min(norms))

    # 
    for n in range(Zib.shape[1]):

        plt.plot(c.contours[n][:,0],
                 c.contours[n][:,1],
                 #color=cmap(n),
                 c='black',
                 linewidth=3,
                 alpha=norms[n]
                )
    plt.xlim(0,512)
    plt.ylim(0,512)
    ctr2+=1
    
plt.show()


ensemble stack:  (8, 3716)
# of occurances: of ensemble over threshold (3366,)
norms:  (531,) Zib:  (3716, 531)
ensemble stack:  (8, 3716)
# of occurances: of ensemble over threshold (2908,)
norms:  (531,) Zib:  (3716, 531)
ensemble stack:  (8, 3716)
# of occurances: of ensemble over threshold (3128,)
norms:  (531,) Zib:  (3716, 531)
ensemble stack:  (8, 3716)
# of occurances: of ensemble over threshold (3176,)
norms:  (531,) Zib:  (3716, 531)
ensemble stack:  (8, 3716)
# of occurances: of ensemble over threshold (3225,)
norms:  (531,) Zib:  (3716, 531)
ensemble stack:  (8, 3716)
# of occurances: of ensemble over threshold (2563,)
norms:  (531,) Zib:  (3716, 531)
ensemble stack:  (8, 3716)
# of occurances: of ensemble over threshold (3130,)
norms:  (531,) Zib:  (3716, 531)
ensemble stack:  (8, 3716)
# of occurances: of ensemble over threshold (3253,)
norms:  (531,) Zib:  (3716, 531)


In [48]:
######################################################
############# VISUALIZE ENSEMBLES OVER TIME ##########
######################################################
selected_ensemble = ens_cs.copy()
ensemble_ids = np.arange(selected_ensemble.shape[0])

ensemble_id = 1
print ("ensemble stack: ", ensemble_times.shape) 
ens_times = ensemble_times[ensemble_id]
print (ens_times.shape)

idx = np.where(ens_times>1000)[0]
print (idx.shape)

cmap = plt.get_cmap("viridis", 531)

for ctr,i in enumerate(idx):
    ax=plt.subplot(5,8,ctr+1)
    activity = c.F_upphase_bin[:,i]
    
    idx2 = np.where(activity>0)[0]
    #print ("idx2: ", idx2)
    for n in idx2:
        plt.plot(c.contours[n][:,0],
                 c.contours[n][:,1],
                 color=cmap(n),
                )
    plt.xlim(0,512)
    plt.ylim(0,512)
    plt.title(str(i))
plt.show()

        

ensemble stack:  (25, 3716)
(3716,)
(86,)


ValueError: num must be 1 <= num <= 40, not 41

In [44]:
#

import networkx as nx

    
def load_correlation_matrix(fname):

    cc = np.load(fname)
    print (cc.shape)
    print (cc[0])

    n_neurons = int(max(int(np.max(cc[:,0])),int(np.max(cc[:,1]))))+1
    cm = np.zeros((n_neurons, n_neurons))

    #
    for k in trange(cc.shape[0]):
        if cc[k,3]<1E-1:
            cm[int(cc[k,0]), int(cc[k,1])] = cc[k,2]

    cthresh = 0.3
    idx = np.where(cm>=cthresh)
    cm  =cm *0
    cm[idx] = 1

    # copy matrix
    cm = cm+cm.T
    
    return cm
    
#
def generate_graph(cm):
    adjacency_matrix = cm.copy()
    rows, cols = np.where(adjacency_matrix == 1)
    edges = zip(rows.tolist(), cols.tolist())
    G = nx.Graph()
    G.add_edges_from(edges)
    
    return G
    
    #nx.draw(gr, node_size=50, 
    #       # labels=mylabels, 
    #       # with_labels=True
    #       )
    #plt.show()
    
def get_degree_distribution(G):
    degrees = []
    for d in G.degree:
        degrees.append(d)

    # 
    degrees  = np.vstack(degrees)
    idx = np.argsort(degrees[:,0])
    degrees = degrees[idx]
    print (degrees)

    ds = degrees[:,1]
    y = np.histogram(ds, bins=np.arange(0,200,2))

    
    return y[1][:-1],y[0]


def get_animal_sessions(animal_id):
    df = pd.read_excel('/media/cat/4TB/donato/CA3 Wheel Animals Database.xlsx')

    #
    binarization_method='upphase'

    idx = np.where(df['Mouse_id']==animal_id)[0].squeeze()
    sessions = df.iloc[idx]['Session_ids'].split(',')
    print ("sessions: ", sessions)
    #
    cmap = plt.get_cmap("viridis", 10)
    ctr=0
    for k in range(len(sessions)):
        sessions[k] = sessions[k].replace(' ','')


    return sessions

####################
root_dir = '/media/cat/4TB/donato/'
animal_ids = [
    'DON-006084',
    'DON-006085',
    'DON-006087',
    'DON-003343',
    'DON-003165',
    'DON-002865'
]


cmap = plt.get_cmap("viridis", 10)
fig=plt.figure()

percentages = []
ave_degrees = []
mean_degree = []

for ctra, animal_id in enumerate(animal_ids):
    ax=plt.subplot(2,3,ctra+1)
    sessions = get_animal_sessions(animal_id)
    res = []
    mean_degree.append([])
    for ctr,session in enumerate(sessions):
        fname = os.path.join(root_dir,
                            animal_id, 
                            session,
                            'suite2p/plane0/cell_correlations.npy')
        #print ("fname", fname)
        cm = load_correlation_matrix(fname)
        G = generate_graph(cm)
        x, y = get_degree_distribution(G)
        plt.plot(x,y,
                color=cmap(ctr))
        res.append(y)
        
        #mean_degree[ctra].append(np.mean(y))
        mean_degree[ctra].append(np.median(y))
        
    y = np.median(np.vstack(res),axis=0)
    plt.plot(x,y,
                color='red',
            linewidth=3)
    ave_degrees.append(y)

   
    #
    plt.title(animal_id)
    if ctra%3==0:
        plt.ylabel("# of neurons")
    if ctr//3==1:
        plt.xlabel("degree")
    plt.xlim(1,200)
    plt.ylim(1,700)
    #
    plt.semilogx()
    plt.semilogy()
    
    ############################
    ax2 = ax.twinx()
    # 
    cumsum = np.cumsum(y)
    cumsum = cumsum/np.max(cumsum)
    percentages.append(cumsum)
    plt.plot(x,cumsum,
                color='black',
            linewidth=3)
    ax2.set_yticks(np.arange(0,1.1,.1))
    plt.ylim(0,1)
    ax2.set_ylabel("Percentage of neurons")
    #ax2.semilogy()
    
plt.show()
    

sessions:  ['20210519', ' 20210520', ' 20210521', ' 20210522', ' 20210523', ' 20210524', ' 20210525', ' 20210526', ' 20210527', ' 20210528']
(140715, 4)
[ 0.00000000e+00  1.00000000e+00 -4.53414669e-02  9.12829711e-27]


100%|███████████████████████████████████████████████████████████| 140715/140715 [00:00<00:00, 535537.08it/s]


[[  0 126]
 [  1  18]
 [  2   3]
 ...
 [528  23]
 [529  54]
 [530   7]]
(170236, 4)
[0.00000000e+000 1.00000000e+000 1.22255035e-001 1.49726929e-184]


100%|███████████████████████████████████████████████████████████| 170236/170236 [00:00<00:00, 474218.33it/s]


[[  0  76]
 [  1  24]
 [  2   3]
 ...
 [581 108]
 [582 100]
 [583 218]]
(110215, 4)
[ 0.00000000e+00  1.00000000e+00 -8.31161700e-02  5.06229971e-86]


...

[[  0  39]
 [  1  21]
 [  2  22]
 ...
 [523  43]
 [524  21]
 [525  49]]
(299151, 4)
[ 0.0000000e+00  1.0000000e+00 -2.8946422e-02  8.1815121e-12]


100%|███████████████████████████████████████████████████████████| 299151/299151 [00:00<00:00, 531253.18it/s]


[[  0  20]
 [  1   8]
 [  2   4]
 ...
 [771  39]
 [772  14]
 [773  47]]
(406351, 4)
[0.         1.         0.39289939 0.        ]


100%|███████████████████████████████████████████████████████████| 406351/406351 [00:00<00:00, 608945.18it/s]


[[  0 130]
 [  1 192]
 [  2 127]
 ...
 [899 523]
 [900  35]
 [901  37]]


In [35]:
clrs = ['red','blue']
fig=plt.figure()
ax=plt.subplot(1,2,1)
for k in range(len(percentages)):
    plt.plot(percentages[k],
             color = clrs[k//3])
plt.ylabel("Cumulative % of cells")
plt.xlabel("Network degree")
#plt.semilogy()
#plt.semilogx()
#
ax=plt.subplot(1,2,2)
for k in range(len(percentages)):
    plt.plot(ave_degrees[k],
             color = clrs[k//3])
plt.ylabel("# of cells")
plt.xlabel("Network degree")

#plt.semilogy()
#plt.semilogx()

plt.show()



In [45]:

#
fig=plt.figure()
for k in range(len(mean_degree)):
    plt.plot(np.arange(10),mean_degree[k],
            color = clrs[k//3])
    
    
    
plt.show()


[[  0  76]
 [  1  24]
 [  2   3]
 ...
 [581 108]
 [582 100]
 [583 218]]
